# Exploring the MovieLens Dataset with `pySpark`

*Apache Spark* is a popular framework for big data. It supports a wide variety of data analytics tasks including data cleaning, stream processing, and machine learning. It can be used to perform large and parallel computations, performing calculations on a single laptop or a cluster of computers. This makes it a useful tool when the data gets too large for `pandas` to handle. Spark can be accessed in Scala, its native API, as well as in Python, Java, R and SQL. In general the `DataFrame` API in Python should achieve the same performance as in Scala. 

This Jupyter Notebook will demonstrate how to get started using `pySpark` and the `DataFrame` API to perform some basic data analysis, including:
- reading in data
- performing aggregations and joins using the Spark SQL module
- calculating summary statistics

We will use the [MovieLens 20M Dataset](https://grouplens.org/datasets/movielens/) on movie ratings to find out:
- What are the most popular movies?
- What are the top rated movies?
- Which movies are the most polarising?

**Note**: This Notebook assumes that you have pySpark installed and configured to work with the Jupyter Notebook. The purpose of this Notebook is to demonstrate some basic Spark techniques rather than to provide an installation guide. For information on how to get pySpark running on the Jupyter Notebook, please refer to [this blog post](https://blog.sicara.com/get-started-pyspark-jupyter-guide-tutorial-ae2fe84f594f).

## Initialising Spark

To use Spark, we must first initialise a `SparkSession`. This is the entry point to using Spark in an application.

Depending on how we have configured Spark, we may also need to use the `findspark` package to make the `SparkSession` available. 

In [1]:
import findspark
findspark.init()

In [2]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('MovieLens').getOrCreate()

## Reading in the data

The *MovieLens 20M* Dataset contains 20,000,263 ratings and 465,564 tag applications across 27,278 movies. The dataset was generated in 2016. All users in the dataset rated at least 20 movies. 

The dataset contains six CSV files. We will be using the **`movies`** and **`ratings`** files. Let's see what these two files look like.

To read in a CSV file, we access the `DataFrameReader` class through `read` and then call the `csv()` method on it. We also specify `option("header", "true")` so that the first row of the file is used for the column headers. 

In [3]:
ratings = spark.read.option("header", "true").csv("ml-20m/ratings.csv")
ratings.show(5)

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|      2|   3.5|1112486027|
|     1|     29|   3.5|1112484676|
|     1|     32|   3.5|1112484819|
|     1|     47|   3.5|1112484727|
|     1|     50|   3.5|1112484580|
+------+-------+------+----------+
only showing top 5 rows



Each row of the `ratings` DataFrame represents one rating for one movie (`movieId`) by one user (`userId`). The ratings use a 5-star scale with half-star increments from 0.5 stars up to 5.0 stars. We can print the DataFrame's column names and types using the `printSchema()` method.

In [4]:
ratings.printSchema()

root
 |-- userId: string (nullable = true)
 |-- movieId: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- timestamp: string (nullable = true)



In [5]:
movies = spark.read.option("header", "true").csv("ml-20m/movies.csv")
movies.show(5)

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows



Each row of the `movies` DataFrame represents one movie and its title and genre(s), indexed by the key `movieId`. We will use this DataFrame to get the movie titles out so we know which movie the ratings in the `ratings` DataFrame are actually referring to. 

## Most popular movies

To get the most popular movies, we are looking for the movies with the highest number of ratings (we use the number of ratings as a proxy for the number of views). To do this, we will perform the following *transformations* on the `ratings` DataFrame: 
- group by `movieId`
- count the number of users (`userId`) associated with each movie 
- rename this column to `num_ratings`
- sort by `num_ratings` in descending order 

In the next cell, we perform these transformations in `pySpark` and store the DataFrame as `most_popular`.

In [6]:
from pyspark.sql.functions import *

most_popular = ratings\
.groupBy("movieId")\
.agg(count("userId"))\
.withColumnRenamed("count(userId)", "num_ratings")\
.sort(desc("num_ratings"))

The DataFrame methods we have used here are:
- `groupBy` - groups the DataFrame by the given column
- `agg` - allows us to perform an aggregate calculation on grouped data (this can be a built-in aggregation function such as *count* or a user defined function)
- `withColumnRenamed` - renames an existing column with a new column name
- `sort` - sorts by the specified column(s)

Because transformations are *lazy* in Spark, the transformations above aren't performed until we call an *action*, such as `show()`, `take()`, or `collect()`.

In [7]:
most_popular.show(10)

+-------+-----------+
|movieId|num_ratings|
+-------+-----------+
|    296|      67310|
|    356|      66172|
|    318|      63366|
|    593|      63299|
|    480|      59715|
|    260|      54502|
|    110|      53769|
|    589|      52244|
|   2571|      51334|
|    527|      50054|
+-------+-----------+
only showing top 10 rows



This DataFrame contains only the `movieId` and `num_ratings`. The actual title of the movie is stored in the `movies` DataFrame. To get the movie titles, we can join our `most_popular` DataFrame with the `movies` DataFrame on `movieId`. By default, `join` performs an inner join which is what we want in this case.

In [8]:
most_popular_movies = most_popular.join(movies, most_popular.movieId == movies.movieId)
most_popular_movies.show(20, truncate=False)

+-------+-----------+-------+-----------------------------------------------------+-------------------------------------------+
|movieId|num_ratings|movieId|title                                                |genres                                     |
+-------+-----------+-------+-----------------------------------------------------+-------------------------------------------+
|296    |67310      |296    |Pulp Fiction (1994)                                  |Comedy|Crime|Drama|Thriller                |
|356    |66172      |356    |Forrest Gump (1994)                                  |Comedy|Drama|Romance|War                   |
|318    |63366      |318    |Shawshank Redemption, The (1994)                     |Crime|Drama                                |
|593    |63299      |593    |Silence of the Lambs, The (1991)                     |Crime|Horror|Thriller                      |
|480    |59715      |480    |Jurassic Park (1993)                                 |Action|Adventure|Sci-

We now have a list of the most popular (or most rated) movies on the *MovieLens* website. As expected, the titles listed here are indeed all well-known movies.

## Top rated movies

We've got the top 10 most popular movies, but now we want to see which movies are perceived to be the best. To get the top rated movies, we are looking for the movies with the highest average rating. To do this, we will use the `ratings` DataFrame and: 

- group by `movieId` 
- calculate the average rating for each movie
- rename this column to `avg_rating`
- sort by `avg_rating` in descending order 

In [9]:
top_rated = ratings\
.groupBy("movieId")\
.agg(avg(col("rating")))\
.withColumnRenamed("avg(rating)", "avg_rating")\
.sort(desc("avg_rating"))

We will again join this DataFrame with the `movies` DataFrame so we know which movie each `movieId` is referring to.

In [10]:
top_rated_movies = top_rated.join(movies, top_rated.movieId == movies.movieId)
top_rated_movies.show(10)

+-------+----------+-------+--------------------+------------------+
|movieId|avg_rating|movieId|               title|            genres|
+-------+----------+-------+--------------------+------------------+
|  95517|       5.0|  95517|Barchester Chroni...|             Drama|
| 109715|       5.0| 109715|Inquire Within (2...|            Comedy|
| 111548|       5.0| 111548|Welcome to Austra...|       Documentary|
| 129905|       5.0| 129905|The Floating Cast...|      Comedy|Drama|
|  98761|       5.0|  98761|Shaolin Temple 2:...|     Action|Comedy|
|  27914|       5.0|  27914|Hijacking Catastr...|       Documentary|
| 129305|       5.0| 129305|Pretty Things (2001)|             Drama|
| 106113|       5.0| 106113|Such Hawks Such H...|       Documentary|
|  94431|       5.0|  94431|Ella Lola, a la T...|(no genres listed)|
|  93707|       5.0|  93707|Prom Queen: The M...|      Comedy|Drama|
+-------+----------+-------+--------------------+------------------+
only showing top 10 rows



The movies listed here appear to be quite niche. We want to focus on top rated movies that also have a decent number of ratings, so want to take into account both the average rating *and* the number of ratings. We can easily create a DataFrame which has both of these columns by specifying multiple expressions within one `agg()` call. 

In [11]:
top_rated = ratings\
.groupBy("movieId")\
.agg(count("userId"), avg(col("rating")))\
.withColumnRenamed("count(userId)", "num_ratings")\
.withColumnRenamed("avg(rating)", "avg_rating")

In [12]:
top_rated_movies = top_rated.join(movies, top_rated.movieId == movies.movieId).sort(desc("avg_rating"), desc("num_ratings"))
top_rated_movies.show(10)

+-------+-----------+----------+-------+--------------------+-------------+
|movieId|num_ratings|avg_rating|movieId|               title|       genres|
+-------+-----------+----------+-------+--------------------+-------------+
| 108527|          2|       5.0| 108527|  Catastroika (2012)|  Documentary|
| 103871|          2|       5.0| 103871|Consuming Kids: T...|  Documentary|
| 106113|          1|       5.0| 106113|Such Hawks Such H...|  Documentary|
|  93707|          1|       5.0|  93707|Prom Queen: The M...| Comedy|Drama|
| 109715|          1|       5.0| 109715|Inquire Within (2...|       Comedy|
| 129905|          1|       5.0| 129905|The Floating Cast...| Comedy|Drama|
| 129305|          1|       5.0| 129305|Pretty Things (2001)|        Drama|
|  98761|          1|       5.0|  98761|Shaolin Temple 2:...|Action|Comedy|
| 111548|          1|       5.0| 111548|Welcome to Austra...|  Documentary|
|  27914|          1|       5.0|  27914|Hijacking Catastr...|  Documentary|
+-------+---

We see that all of the movies with an average rating of exactly 5.0 have 2 or less ratings. We would like to only consider movies that have achieved some minimum number of ratings. To determine an appropriate threshold, we should investigate the distribution of `num_ratings`. We can do this by calculating some summary statistics within Spark.

In [13]:
# Calculate average, minimum, and maximum of num_ratings
top_rated_movies.select([mean('num_ratings'), min('num_ratings'), max('num_ratings')]).show(1)

+-----------------+----------------+----------------+
| avg(num_ratings)|min(num_ratings)|max(num_ratings)|
+-----------------+----------------+----------------+
|747.8411232425965|               1|           67310|
+-----------------+----------------+----------------+



To calculate quantiles we use the `approxQuantile` method. This method can calculate the quantiles of the specified column approximately or exactly, depending on the value of the relative error parameter. If the relative error parameter is set to 0 then the quantiles are calculated exactly, however this can be expensive. 

In [14]:
# median
top_rated_movies.approxQuantile('num_ratings', [0.5], 0)

[18.0]

In [15]:
# first quartile
top_rated_movies.approxQuantile('num_ratings', [0.25], 0)

[3.0]

In [16]:
# third quartile
top_rated_movies.approxQuantile('num_ratings', [0.75], 0)

[205.0]

The mean is much greater than the median value, suggesting that this distribution is skewed to the right. We will choose a minimum threshold of 500 ratings, however there is no right or wrong answer here and the reader is encouraged to experiment with different values for this threshold.

In [17]:
top_rated_movies.where("num_ratings > 500").show(20, truncate=False)

+-------+-----------+------------------+-------+---------------------------------------------------------------------------+-----------------------------------------+
|movieId|num_ratings|avg_rating        |movieId|title                                                                      |genres                                   |
+-------+-----------+------------------+-------+---------------------------------------------------------------------------+-----------------------------------------+
|318    |63366      |4.446990499637029 |318    |Shawshank Redemption, The (1994)                                           |Crime|Drama                              |
|858    |41355      |4.364732196832306 |858    |Godfather, The (1972)                                                      |Crime|Drama                              |
|50     |47006      |4.334372207803259 |50     |Usual Suspects, The (1995)                                                 |Crime|Mystery|Thriller                   

We've now gotten a list of the top rated movies on MovieLens, which includes the usual movies considered to be all time greats such as *The Shawshank Redemption* and *Casablanca*. Interestingly, nearly all of these movies appear in the [top 100 of the IMDb top rated movies list](https://www.imdb.com/chart/top) as well, with the exception of the *The Third Man* (listed as #135) and *Band of Brothers* which is technically a TV series rather than a movie.

What's also interesting is that this list of movies is not the same as the list of the most popular movies. *The Shawshank Redemption*, *Schindler's List*, and *The Usual Suspects* were all popular movies which also appear in this list. However, other movies such as *Pulp Fiction*, *Forrest Gump*, and *The Silence of the Lambs* made the top 10 most popular but not the top 10 (or even top 20) most rated. This suggests that some movies actually divide opinion.

## Most polarising movies (Marmite movies)

Next, we will try to answer the question, *What are the most polarising movies*? These are the movies that divide opinon, with people tending to rate them either really highly or really poorly. We will refer to these as *Marmite* movies. Again, we only want to consider movies that achieve some minimum number of ratings - we will stick with our previous threshold of 500 ratings. 

To approach this, we will look for the movies with the highest standard deviation in rating. This is a measure of how much the data varies from the mean, so in this case, how much a movie's ratings vary around its mean rating. A high standard deviation would suggest that the movie's ratings are highly variable. There are other approaches to this as well, for instance, what proportion of the ratings are very positive or very negative.

In [18]:
ratings_stddev = ratings\
.groupBy("movieId")\
.agg(count("userId").alias("num_ratings"), 
     avg(col("rating")).alias("avg_rating"),
     stddev(col("rating")).alias("std_rating")
    )\
.where("num_ratings > 500")

In [19]:
ratings_stddev.show(5)

+-------+-----------+------------------+------------------+
|movieId|num_ratings|        avg_rating|        std_rating|
+-------+-----------+------------------+------------------+
|    296|      67310| 4.174231169217055|0.9760762295742448|
|   1090|      15808| 3.919977226720648|0.8272067263021856|
|   3959|       2869| 3.699372603694667|0.8607671626686735|
|   2294|      10163| 3.303207714257601|0.9047000233824075|
|   6731|       1173|3.5571184995737424|0.9189292350434509|
+-------+-----------+------------------+------------------+
only showing top 5 rows



In [20]:
marmite_movies = ratings_stddev.join(movies, ratings_stddev.movieId == movies.movieId)

In [22]:
marmite_movies.sort(desc("std_rating")).show(15, truncate=False)

+-------+-----------+------------------+------------------+-------+----------------------------------------------------------------------------+-------------------------------------+
|movieId|num_ratings|avg_rating        |std_rating        |movieId|title                                                                       |genres                               |
+-------+-----------+------------------+------------------+-------+----------------------------------------------------------------------------+-------------------------------------+
|27899  |579        |2.8860103626943006|1.4221290413577283|27899  |What the #$*! Do We Know!? (a.k.a. What the Bleep Do We Know!?) (2004)      |Comedy|Documentary|Drama             |
|1924   |2304       |2.6319444444444446|1.4201711823223824|1924   |Plan 9 from Outer Space (1959)                                              |Horror|Sci-Fi                        |
|91104  |516        |2.4728682170542635|1.353614474548174 |91104  |Twilight Saga: Bre

We see that the list of polarising movies includes the *Twilight* movies, the controversial *Passion of the Christ*, and the cult low-budget science fiction movie *Plan 9 from Outer Space*.

## Conclusion

This tutorial has demonstrated how to use the `pySpark DataFrame` API to perform some simple data analysis tasks. In particular, we have seen how to perform aggregations, joins, and compute summary statistics on large datasets. There is a lot more that could be done with this dataset, including investigating other ways to identify polarising movies, looking at the effect of movie genres, and building a recommender system. Note that when working in `pySpark`, it may useful to refer back to the [official pySpark documentation](https://spark.apache.org/docs/latest/api/python/pyspark.html). 